In [1]:
BASE_DIR = ""

# Colab 환경 초기화 
* local에서 실행시 불필요함

In [2]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [3]:
!pip install openai
!pip install streamlit
!pip install datasets
!pip install pinecone-client
!pip install tiktoken
!pip install PyPDF2
!pip install sentence-transformers
!pip install evaluate

In [4]:
###########################################
# 1-1. Google drive mount

if IN_COLAB == True:
    from google.colab import drive
    drive.mount('/content/drive')
    
    BASE_DIR = "/content/drive/MyDrive/Colab Notebooks/quick-start-guide-to-llms/notebooks/"

In [5]:
!cd "/content/drive/MyDrive/Colab Notebooks/quick-start-guide-to-llms/notebooks"

zsh:cd:1: no such file or directory: /content/drive/MyDrive/Colab Notebooks/quick-start-guide-to-llms/notebooks


이 노트북은 최신 openai 패키지 버전을 사용하도록 업데이트되었습니다! 당시 1.6.1

# 2장: 독점 모델로 애플리케이션 시작하기
* 독점 모델 개요
* OpenAI + 임베딩 / GPT3 / ChatGPT 소개
* 벡터 데이터베이스 소개
* 벡터 데이터베이스, BERT 및 GPT3로 신경/의미 정보 검색 시스템 구축하기

In [6]:
from openai import OpenAI
from datetime import datetime
import hashlib
import re
import os
from tqdm import tqdm
import numpy as np

import logging

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)


In [7]:
if IN_COLAB == True:
    pinecone_key = "PINECONE API KEY"
    openai_key="OPENAI API KEY"
else:
    pinecone_key = os.environ.get('PINECONE_API_KEY')
    openai_key=os.environ.get("OPENAI_API_KEY")
    
client = OpenAI(
    api_key=openai_key
)

INDEX_NAME = 'semantic-search'
NAMESPACE = 'default'
ENGINE = 'text-embedding-ada-002'

In [9]:
# 기존 소스 오류 수정
# import pinecone
# pinecone.init(api_key=pinecone_key, environment="us-west1-gcp")

from pinecone import Pinecone, PodSpec
pinecone = Pinecone(api_key=pinecone_key)

In [10]:
# OpenAI API에서 임베딩 목록을 가져오는 헬퍼 함수
def get_embeddings(texts, engine=ENGINE):
    response = client.embeddings.create(
        input=texts,
        model=engine
    )
    
    return [d.embedding for d in list(response.data)]

def get_embedding(text, engine=ENGINE):
    return get_embeddings([text], engine)[0]
    
len(get_embedding('hi')), len(get_embeddings(['hi', 'hello']))

(1536, 2)

In [13]:
# 기존 소스 오류 수정
# https://docs.pinecone.io/docs/view-index-information
# https://docs.pinecone.io/docs/create-an-index

if not INDEX_NAME in pinecone.list_indexes().names():
    pinecone.create_index(
        INDEX_NAME, # 인덱스 이름
        dimension=1536, # 벡터의 치수
        metric='cosine', # 인덱스를 검색할 때 사용할 유사성 메트릭
        spec=PodSpec(
          environment="gcp-starter"
        )
        # pod_type="p1" # 파인콘 파드의 유형
    )

# 인덱스를 변수로 저장
index = pinecone.Index(INDEX_NAME)

In [14]:
def my_hash(s):
    # 입력 문자열의 MD5 해시를 16진수 문자열로 반환합니다.
    return hashlib.md5(s.encode()).hexdigest()

my_hash('I love to hash it')

'ae76cc4dfd345ecaeea9b8ba0d5c3437'

In [15]:
def prepare_for_pinecone(text, engine=ENGINE):
    # 현재 UTC 날짜 및 시간을 가져옵니다.
    now = datetime.utcnow()
    
    # 지정된 엔진을 사용하여 입력 목록의 각 문자열에 대한 벡터 임베딩을 생성합니다.
    embeddings = get_embeddings(texts, engine=engine)
    
    # 각 입력 문자열과 해당 벡터 임베딩에 대해 (해시, 임베딩, 메타데이터)의 튜플을 생성합니다.
    # my_hash() 함수는 각 문자열에 대해 고유한 해시를 생성하는 데 사용되며, datetime.utcnow() 함수는 현재 UTC 날짜 및 시간을 생성하는 데 사용됩니다.
    return [
        (
            my_hash(text), # my_hash() 함수를 사용하여 생성된 각 문자열에 대한 고유 ID
            embedding, # 문자열의 벡터 임베딩
            dict(text=text, date_uploaded=now) # 원본 텍스트와 현재 UTC 날짜 및 시간을 포함한 메타데이터의 사전입니다.
        ) 
        for text, embedding in zip(texts, embeddings) # 각 입력 문자열과 해당 벡터 임베딩에 대해 반복합니다.
    ]

In [16]:
texts = ['hi']

In [17]:
prepare_for_pinecone(texts)[0]

('49f68a5c8493ec2c0bf489821c21fc3b',
 [-0.030913319438695908,
  -0.020414210855960846,
  -0.019505759701132774,
  -0.04178878664970398,
  -0.024813713505864143,
  0.024307576939463615,
  -0.0179743692278862,
  -0.017701834440231323,
  -0.0065019200555980206,
  -0.015910886228084564,
  0.025890879333019257,
  -0.006949656642973423,
  -0.01790948025882244,
  -0.011848808266222477,
  0.011465960182249546,
  0.01648191176354885,
  0.038751959800720215,
  0.0005187098286114633,
  0.03221110627055168,
  -0.008701670914888382,
  -0.019635537639260292,
  -0.0049056401476264,
  -0.009298654273152351,
  -0.014327583834528923,
  -0.022867031395435333,
  0.002483642427250743,
  0.010051371529698372,
  -0.01176445186138153,
  0.0026069325394928455,
  -0.026020657271146774,
  0.014535229653120041,
  0.0006987779634073377,
  -0.035767048597335815,
  -0.014963500201702118,
  -0.009486833587288857,
  -0.024748824536800385,
  0.006988590583205223,
  -0.02111501805484295,
  0.01918131299316883,
  -0.0056

In [18]:
_id, embedding, metadata = prepare_for_pinecone(texts)[0]

print('ID:  ',_id, '\nLEN: ', len(embedding), '\nMETA:', metadata)

ID:   49f68a5c8493ec2c0bf489821c21fc3b 
LEN:  1536 
META: {'text': 'hi', 'date_uploaded': datetime.datetime(2024, 3, 7, 5, 21, 30, 973413)}


In [19]:
def upload_texts_to_pinecone(texts, namespace=NAMESPACE, batch_size=None, show_progress_bar=False):
    # prepare_for_pinecone 함수를 호출하여 인덱싱을 위해 입력 텍스트를 준비합니다.
    total_upserted = 0
    if not batch_size:
        batch_size = len(texts)

    _range = range(0, len(texts), batch_size)
    for i in tqdm(_range) if show_progress_bar else _range:
        batch = texts[i: i + batch_size]
        prepared_texts = prepare_for_pinecone(batch)

        # 인덱스 객체의 upsert() 메서드를 사용하여 준비된 텍스트를 Pinecone에 업로드합니다.
        total_upserted += index.upsert(
            prepared_texts,
            namespace=namespace
        )['upserted_count']

    return total_upserted

# 입력 텍스트로 upload_texts_to_pinecone() 함수를 호출합니다.
upload_texts_to_pinecone(texts)

1

In [20]:
def query_from_pinecone(query, top_k=3):
    # 문서와 동일한 임베더에서 임베딩 받기
    query_embedding = get_embedding(query, engine=ENGINE)
    # query_embedding = get_embedding(query)

    return index.query(
      vector=query_embedding,
      top_k=top_k,
      namespace=NAMESPACE,
      include_metadata=True   # gets the metadata (dates, text, etc)
    ).get('matches')

query_from_pinecone('hello')

[]

In [21]:
import hashlib

def delete_texts_from_pinecone(texts, namespace=NAMESPACE):
    # 각 텍스트의 해시(ID)를 계산합니다.
    hashes = [hashlib.md5(text.encode()).hexdigest() for text in texts]
    
    # ids 매개변수는 삭제할 ID(해시) 목록을 지정하는 데 사용됩니다.
    return index.delete(ids=hashes, namespace=namespace)

# 텍스트 삭제
delete_texts_from_pinecone(texts)

# 인덱스가 비어 있는지 테스트
query_from_pinecone('hello')

[]

In [22]:
# 틱토큰 라이브러리 가져오기
import tiktoken

# 'cl100k_base' 모델에 대한 토큰화 도구 초기화하기
# 이 토큰화 도구는 'ada-002' 임베딩 모델과 함께 작동하도록 설계되었습니다.
tokenizer = tiktoken.get_encoding("cl100k_base")

# 토큰라이저를 사용하여 'hey there' 텍스트를 인코딩하기
# 결과 출력은 인코딩된 텍스트를 나타내는 정수 목록입니다.
# 'ada-002' 모델을 사용하여 임베딩하는 데 필요한 입력 형식입니다.
tokenizer.encode('hey there')

[36661, 1070]

In [23]:
# 텍스트를 최대 토큰 수의 청크로 분할하는 함수입니다. OpenAI에서 영감을 얻음
def overlapping_chunks(text, max_tokens = 500, overlapping_factor = 5):
    '''
    max_tokens: 청크당 원하는 토큰 수
    overlapping_factor: 이전 청크와 겹치는 각 청크를 시작할 문장 수
    '''

    # 문장 부호를 사용하여 텍스트 분할
    sentences = re.split(r'[.?!]', text)

    # 각 문장의 토큰 개수 가져오기
    n_tokens = [len(tokenizer.encode(" " + sentence)) for sentence in sentences]
    
    chunks, tokens_so_far, chunk = [], 0, []

    # 튜플로 결합된 문장과 토큰을 반복합니다.
    for sentence, token in zip(sentences, n_tokens):

        # 지금까지의 토큰 수에 현재 문장의 토큰 수를 더한 수가 
        # 최대 토큰 수보다 크면 청크 목록에 청크를 추가하고 재설정합니다.
        # 청크와 지금까지의 토큰을 재설정합니다.
        if tokens_so_far + token > max_tokens:
            chunks.append(". ".join(chunk) + ".")
            if overlapping_factor > 0:
                chunk = chunk[-overlapping_factor:]
                tokens_so_far = sum([len(tokenizer.encode(c)) for c in chunk])
            else:
                chunk = []
                tokens_so_far = 0

        # 현재 문장의 토큰 수가 최대 토큰 수보다 많으면 
        # 토큰 수보다 많으면 다음 문장으로 이동합니다.
        if token > max_tokens:
            continue

        # 그렇지 않으면, 청크에 문장을 추가하고 토큰 수를 합산합니다.
        chunk.append(sentence)
        tokens_so_far += token + 1
    if chunk:
        chunks.append(". ".join(chunk) + ".")

    return chunks

In [24]:
import PyPDF2

# 읽기 바이너리 모드로 PDF 파일 열기
with open(BASE_DIR + '../data/pds2.pdf', 'rb') as file:

    # PDF 리더 객체를 만듭니다.
    reader = PyPDF2.PdfReader(file)

    # 텍스트를 담을 빈 문자열을 초기화합니다.
    principles_of_ds = ''
    # PDF 파일의 각 페이지를 반복합니다.
    for page in tqdm(reader.pages):
        text = page.extract_text()
        principles_of_ds += '\n\n' + text[text.find(' ]')+2:]

# PDF 파일의 모든 텍스트가 포함된 최종 문자열을 인쇄합니다.
principles_of_ds = principles_of_ds.strip()

print(len(principles_of_ds))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 428/428 [04:55<00:00,  1.45it/s]

575490


In [25]:
from urllib.request import urlopen

# 곤충에 관한 교과서
text = urlopen('https://www.gutenberg.org/cache/epub/10834/pg10834.txt').read().decode()

In [26]:
split = overlapping_chunks(principles_of_ds, overlapping_factor=0)
avg_length = sum([len(tokenizer.encode(t)) for t in split]) / len(split)
print(f'non-overlapping chunking approach has {len(split)} documents with average length {avg_length:.1f} tokens')

non-overlapping chunking approach has 286 documents with average length 474.1 tokens


In [27]:
split = overlapping_chunks(principles_of_ds)
avg_length = sum([len(tokenizer.encode(t)) for t in split]) / len(split)
print(f'overlapping chunking approach has {len(split)} documents with average length {avg_length:.1f} tokens')

overlapping chunking approach has 392 documents with average length 485.2 tokens


In [28]:
# 카운터 및 re 라이브러리 가져오기
from collections import Counter
import re

# 'principles_of_ds'에서 하나 이상의 공백이 있는 모든 항목 찾기
matches = re.findall(r'[\s]{1,}', principles_of_ds)

# 문서에서 가장 빈번하게 발생하는 공백 10가지
most_common_spaces = Counter(matches).most_common(10)

# 가장 일반적인 공백과 그 빈도를 인쇄합니다.
print(most_common_spaces)


[(' ', 82259), ('\n', 9220), ('  ', 1592), ('\n\n', 333), ('\n   ', 250), ('\n\n\n', 82), ('\n    ', 73), ('\n ', 46), (' \n', 39), ('     ', 34)]


In [29]:
# Only keep documents of at least 100 characters split by a custom delimiter
split = list(filter(lambda x: len(x) > 50, principles_of_ds.split('\n\n')))

avg_length = sum([len(tokenizer.encode(t)) for t in split]) / len(split)
print(f'custom delimiter approach has {len(split)} documents with average length {avg_length:.1f} tokens')

custom delimiter approach has 426 documents with average length 316.3 tokens


In [30]:
embeddings = None
for s in tqdm(range(0, len(split), 100)):
    if embeddings is None:
        embeddings = np.array(get_embeddings(split[s:s+100], engine=ENGINE))
    else:
        embeddings = np.vstack([embeddings, np.array(get_embeddings(split[s:s+100], engine=ENGINE))])
    

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:07<00:00,  1.60s/it]


In [31]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 'embeddings'라는 텍스트 임베딩 목록이 있다고 가정합니다.
# 먼저, 모든 임베딩 쌍 사이의 코사인 유사도 행렬을 계산합니다.
cosine_sim_matrix = cosine_similarity(embeddings)

# 응집 클러스터링 모델을 인스턴스화합니다.
agg_clustering = AgglomerativeClustering(
    n_clusters=None, # 알고리즘이 데이터를 기반으로 최적의 클러스터 수를 결정합니다.
    distance_threshold=0.1, # 클러스터 간의 모든 쌍별 거리가 0.1보다 커질 때까지 클러스터를 형성합니다.
    # affinity='precomputed', # 미리 계산된 거리 행렬(1 - 유사도 행렬)을 입력으로 제공합니다.
    linkage='complete', # 구성 요소 간의 최대 거리를 기준으로 가장 작은 클러스터를 반복적으로 병합하여 클러스터를 형성합니다.
)

# 코사인 거리 행렬(1 - 유사도 행렬)에 모델을 맞춥니다.
agg_clustering.fit(1 - cosine_sim_matrix)

# 각 임베딩에 대한 클러스터 레이블을 가져옵니다.
cluster_labels = agg_clustering.labels_

# 각 클러스터의 임베딩 개수를 출력합니다.
unique_labels, counts = np.unique(cluster_labels, return_counts=True)
for label, count in zip(unique_labels, counts):
    print(f'Cluster {label}: {count} embeddings')


Cluster 0: 1 embeddings
Cluster 1: 1 embeddings
Cluster 2: 1 embeddings
Cluster 3: 1 embeddings
Cluster 4: 1 embeddings
Cluster 5: 1 embeddings
Cluster 6: 1 embeddings
Cluster 7: 1 embeddings
Cluster 8: 1 embeddings
Cluster 9: 1 embeddings
Cluster 10: 1 embeddings
Cluster 11: 1 embeddings
Cluster 12: 1 embeddings
Cluster 13: 1 embeddings
Cluster 14: 1 embeddings
Cluster 15: 1 embeddings
Cluster 16: 1 embeddings
Cluster 17: 1 embeddings
Cluster 18: 1 embeddings
Cluster 19: 1 embeddings
Cluster 20: 1 embeddings
Cluster 21: 1 embeddings
Cluster 22: 1 embeddings
Cluster 23: 1 embeddings
Cluster 24: 1 embeddings
Cluster 25: 1 embeddings
Cluster 26: 1 embeddings
Cluster 27: 1 embeddings
Cluster 28: 1 embeddings
Cluster 29: 1 embeddings
Cluster 30: 1 embeddings
Cluster 31: 1 embeddings
Cluster 32: 1 embeddings
Cluster 33: 1 embeddings
Cluster 34: 1 embeddings
Cluster 35: 1 embeddings
Cluster 36: 1 embeddings
Cluster 37: 1 embeddings
Cluster 38: 1 embeddings
Cluster 39: 1 embeddings
Cluster 40

In [32]:
pruned_documents = []
for _label, count in zip(unique_labels, counts):
    pruned_documents.append('\n\n'.join([text for text, label in zip(split, cluster_labels) if label == _label]))

    
avg_length = sum([len(tokenizer.encode(t)) for t in pruned_documents]) / len(pruned_documents)
# print(f'Our pruning approach has {len(pruned_documents)} documents with average length {avg_length:.1f} tokens')
print(f'우리의 가지치기 접근 방식에는 평균 길이 {avg_length:.1f} 토큰을 가진 {len(pruned_documents)} 문서가 있습니다.')

우리의 가지치기 접근 방식에는 평균 길이 316.3 토큰을 가진 426 문서가 있습니다.


In [33]:
print(pruned_documents[0])

45, 349, 350
   survey  356, 357, 359, 360, 362, 363
Spark  376
spark_sklearn
   reference link  405
square matrix  77
standard deviation  146
standard normal distribution  133
statistical modeling  226
statistics  137, 201
   measures of center  144
   measures of relative standing  150, 153, 156
   measures of variation  145, 148
   measuring  144
stock price
   example  355
stock prices
   predicting, on social media  338
   text sentiment analysis  338, 339
structured data
   about  33
   versus unstructured data  33
superset  87
supervised learning  215
supervised learning models  271
supervised learning
   classification  219
   regression  219
   types  219
supervised machine learning  224
T
TensorFlow
   about  368, 369, 370, 371, 372, 374
   neural networks  368, 369, 370, 371, 372, 374   using  363, 364, 366, 367
titanic dataset  68
training error
   versus cross-validation error  318, 320
U
underfitting  308
unstructured data
   about  33
   versus structured data  33
unsupe

In [34]:
upload_texts_to_pinecone(pruned_documents, batch_size=128)

4

In [35]:
query = 'How do z scores work?'

results_from_pinecone = query_from_pinecone(query, top_k=5)

for result_from_pinecone in results_from_pinecone:
    print(f"{result_from_pinecone['id']}\t{result_from_pinecone['score']:.2f}\t{result_from_pinecone['metadata']['text'][:50]}")
    

In [36]:
"""
이 예는 의미론적 텍스트 유사성(STS)을 위한 교차 인코더를 사용하여 쿼리와 말뭉치에서 가능한 모든
문장과 의미론적 텍스트 유사성(STS)을 위한 교차 인코더를 사용하여 점수를 계산합니다.
그런 다음 주어진 쿼리에 대해 가장 유사한 문장을 출력합니다.
"""
from sentence_transformers.cross_encoder import CrossEncoder
import numpy as np
from torch import nn

# 사전 학습된 크로스 인코더
# cross_encoder = CrossEncoder('cross-encoder/mmarco-mMiniLMv2-L12-H384-v1')
cross_encoder = CrossEncoder('jeffwan/mmarco-mMiniLMv2-L12-H384-v1')

In [37]:
def get_results_from_pinecone(query, top_k=3, re_rank=False, verbose=True):

    results_from_pinecone = query_from_pinecone(query, top_k=top_k)
    if not results_from_pinecone:
        return []

    if verbose:
        print("Query:", query)
    
    
    final_results = []

    if re_rank:
        if verbose:
            print('Document ID (Hash)\t\tRetrieval Score\tCE Score\tText')

        sentence_combinations = [[query, result_from_pinecone['metadata']['text']] for result_from_pinecone in results_from_pinecone]

        # 이러한 조합에 대한 유사도 점수를 계산합니다.
        similarity_scores = cross_encoder.predict(sentence_combinations, activation_fct=nn.Sigmoid())

        # 점수를 내림차순으로 정렬
        sim_scores_argsort = reversed(np.argsort(similarity_scores))

        # 점수를 인쇄합니다.
        for idx in sim_scores_argsort:
            result_from_pinecone = results_from_pinecone[idx]
            final_results.append(result_from_pinecone)
            if verbose:
                print(f"{result_from_pinecone['id']}\t{result_from_pinecone['score']:.2f}\t{similarity_scores[idx]:.2f}\t{result_from_pinecone['metadata']['text'][:50]}")
        return final_results

    if verbose:
        print('Document ID (Hash)\t\tRetrieval Score\tText')
    for result_from_pinecone in results_from_pinecone:
        final_results.append(result_from_pinecone)
        if verbose:
            print(f"{result_from_pinecone['id']}\t{result_from_pinecone['score']:.2f}\t{result_from_pinecone['metadata']['text'][:50]}")

    return final_results

In [38]:
final_results = get_results_from_pinecone(query, top_k=3, re_rank=True)

Query: How do z scores work?
Document ID (Hash)		Retrieval Score	CE Score	Text


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

49f68a5c8493ec2c0bf489821c21fc3b	0.75	0.00	hi


In [39]:
final_results = get_results_from_pinecone(query, top_k=10, re_rank=True)

Query: How do z scores work?
Document ID (Hash)		Retrieval Score	CE Score	Text


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

49f68a5c8493ec2c0bf489821c21fc3b	0.75	0.00	hi


In [40]:
delete_texts_from_pinecone(pruned_documents)

{}

# BoolQ

In [41]:
from datasets import load_dataset
from evaluate import load


dataset = load_dataset("boolq")

In [42]:
dataset['validation'][0]

{'question': 'does ethanol take more energy make that produces',
 'answer': False,
 'passage': "All biomass goes through at least some of these steps: it needs to be grown, collected, dried, fermented, distilled, and burned. All of these steps require resources and an infrastructure. The total amount of energy input into the process compared to the energy released by burning the resulting ethanol fuel is known as the energy balance (or ``energy returned on energy invested''). Figures compiled in a 2007 report by National Geographic Magazine point to modest results for corn ethanol produced in the US: one unit of fossil-fuel energy is required to create 1.3 energy units from the resulting ethanol. The energy balance for sugarcane ethanol produced in Brazil is more favorable, with one unit of fossil-fuel energy required to create 8 from the ethanol. Energy balance estimates are not easily produced, thus numerous such reports have been generated that are contradictory. For instance, a sep

In [43]:
for idx in tqdm(range(0, len(dataset['validation']), 256)):
    data_sample = dataset['validation'][idx:idx + 256]

    passages = data_sample['passage']

    upload_texts_to_pinecone(passages)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:09<00:00,  1.42it/s]


In [44]:
from random import sample

query = sample(dataset['validation']['question'], 1)[0]
print(query)
final_results = get_results_from_pinecone(query, top_k=3, re_rank=True)


does the human body have a cannabinoid system
Query: does the human body have a cannabinoid system
Document ID (Hash)		Retrieval Score	CE Score	Text


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

49f68a5c8493ec2c0bf489821c21fc3b	0.73	0.17	hi


In [45]:
q_to_hash = {data['question']: my_hash(data['passage']) for data in dataset['validation']}

q_to_hash[query]

'7c04981525ffb6cb14f43ec062beafef'

In [46]:
# super_glue_metric = load('super_glue', 'boolq') # 정확도만 확인합니다.

# 1000개의 유효성 검사 데이터 포인트에 대한 성능 재순위를 테스트해 보겠습니다.
# 여기서는 속도를 높이기 위해 Pinecone을 사용할 수 없습니다.
# 하지만 Pinecone으로 파이프라인의 지연 시간을 테스트하기에도 좋은 시기입니다.
val_sample = dataset['validation']#[:1000]

In [47]:
logger.setLevel(logging.CRITICAL)

predictions = []

# Pinecone의 지연 시간이 일관되게 유지되도록 top_k를 동일하게 유지합니다.
# 그리고 유일한 큰 시간 차이는 리랭킹에서 발생합니다.
for question in tqdm(val_sample['question']):
    retrieved_hash = get_results_from_pinecone(question, top_k=1, re_rank=False, verbose=False)[0]['id']
    correct_hash = q_to_hash[question]
    predictions.append(retrieved_hash == correct_hash)
    
accuracy = sum(predictions)/len(predictions)

print(f'Accuracy without re-ranking: {accuracy}')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3270/3270 [27:52<00:00,  1.96it/s]

Accuracy without re-ranking: 0.0


In [ ]:
logger.setLevel(logging.CRITICAL)

predictions = []

# Pinecone의 지연 시간이 일관되게 유지되도록 top_k를 동일하게 유지합니다.
# 그리고 유일한 큰 시간 차이는 리랭킹에서 발생합니다.
for question in tqdm(val_sample['question']):
    retrieved_hash = get_results_from_pinecone(question, top_k=3, re_rank=True, verbose=False)[0]['id']
    correct_hash = q_to_hash[question]
    predictions.append(retrieved_hash == correct_hash)
    
accuracy = sum(predictions)/len(predictions)

print(f'Accuracy with re-ranking: {accuracy}')

 97%|█████████████████████████████████████████████████████████████████████████████████████████▎  | 3173/3270 [29:42<00:48,  1.99it/s]

In [ ]:
# 순위 재조정 시와 그렇지 않은 경우의 시간 차이에 유의하세요.

In [ ]:
def eval_ranking(query, cross_encoder, top_k=3):
    results_from_pinecone = query_from_pinecone(query, top_k=top_k)
    sentence_combinations = [[query, result_from_pinecone['metadata']['text']] for result_from_pinecone in results_from_pinecone]
    similarity_scores = cross_encoder.predict(sentence_combinations)
    sim_scores_argsort = list(reversed(np.argsort(similarity_scores)))
    re_ranked_final_result = results_from_pinecone[sim_scores_argsort[0]]
    return results_from_pinecone[0]['id'], re_ranked_final_result['id']


In [ ]:
# 사전 학습된 다른 크로스 인코더 시도하기
# sentence-transformers/multi-qa-mpnet-base-cos-v1
newer_cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2')

In [ ]:
i = 0
print_every = 50
predictions = []
for question in tqdm(val_sample['question']):
    retrieved_hash, reranked_hash = eval_ranking(question, newer_cross_encoder, top_k=3)
    correct_hash = q_to_hash[question]
    predictions.append((retrieved_hash == correct_hash, reranked_hash == correct_hash))
    i += 1
    if i % print_every == 0:
        print(f'Step {i}')
        raw_accuracy = sum([p[0] for p in predictions])/len(predictions)
        reranked_accuracy = sum([p[1] for p in predictions])/len(predictions)

        print(f'Accuracy without re-ranking: {raw_accuracy}')
        print(f'Accuracy with re-ranking: {reranked_accuracy}')


In [ ]:
raw_accuracy = sum([p[0] for p in predictions])/len(predictions)
reranked_accuracy = sum([p[1] for p in predictions])/len(predictions)

print(f'Using cross-encoder: {newer_cross_encoder.config._name_or_path}')
print(f'Accuracy without re-ranking: {raw_accuracy}')
print(f'Accuracy with re-ranking: {reranked_accuracy}')


# Fine-tuning re-ranker

In [ ]:
# https://github.com/UKPLab/sentence-transformers/blob/master/examples/training/ms_marco/train_cross-encoder_scratch.py

In [ ]:
dataset['train'][0]

In [ ]:
dataset['train'][1]

In [ ]:
from sentence_transformers import InputExample, losses, evaluation
from torch.utils.data import DataLoader
from random import shuffle

shuffled_training_passages = dataset['train']['passage'].copy()
shuffle(shuffled_training_passages)


train_samples = [
  InputExample(texts=[d['question'], d['passage']], label=1) for d in dataset['train']
]

# 부정적인 예제 추가
train_samples += [
  InputExample(texts=[d['question'], shuffled_training_passages[i]], label=0) for i, d in enumerate(dataset['train'])
]

shuffle(train_samples)

# 내 데이터에 과적합의 위험이 있지만 원할 수도 있습니다. 
# 충분한 입력 및 출력 유효성 검사와 결합하면 내 데이터에 과적합한 모델을 사용하여 실행 가능한 제품을 만들 수 있습니다.

In [ ]:
len(train_samples)

In [ ]:
device = torch.device('cpu')
# device = torch.device('cuda')  # NVIDIA GPU

model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2', num_labels=1, device=device)

In [ ]:
train_samples[0].__dict__

In [ ]:
model.predict(train_samples[0].texts, activation_fct=nn.Sigmoid())

In [ ]:
from sentence_transformers.cross_encoder.evaluation import CECorrelationEvaluator, CEBinaryClassificationEvaluator
import math
import torch
from random import sample

logger.setLevel(logging.DEBUG)  # just to get some logs

num_epochs = 2

model_save_path = './fine_tuned_ir_cross_encoder'

# train_samples = sample(train_samples, 1000)

# int(len(train_samples)*.8)
train_dataloader = DataLoader(train_samples[:int(len(train_samples)*.8)], shuffle=True, batch_size=32)

# 훈련 성능을 위한 평가자
evaluator = CEBinaryClassificationEvaluator.from_input_examples(train_samples[-int(len(train_samples)*.8):], name='test')

# 워밍업 단계에 대한 경험 법칙
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)  # 워밍업을 위한 훈련 데이터의 10%
print(f"Warmup-steps: {warmup_steps}")

In [ ]:
# # ##### 모델을 로드하고 테스트 세트에서 평가합니다.
# print(evaluator(model))

# Train the model
model.fit(
    train_dataloader=train_dataloader,
    # loss_fct=losses.nn.CrossEntropyLoss(),
    loss_fct= nn.CrossEntropyLoss(),
    activation_fct=nn.Sigmoid(),
    evaluator=evaluator,
    epochs=num_epochs,
    warmup_steps=warmup_steps,
    output_path=model_save_path,
    # use_amp=True, # GPU 사용시
    use_amp=False,  # CPU 사용시
)

# # #### 모델을 로드하고 테스트 세트에서 평가하기
# print(evaluator(model))


In [ ]:
# 오픈 소스에서도 더 미세 조정된 버전을 실행하여 일치시킬 수 있을까요?
# 여기서 더 잘 작동하는지에 따라 다릅니다.

In [ ]:
finetuned = CrossEncoder(model_save_path)

print(finetuned.predict(['hello', 'hi'], activation_fct=nn.Sigmoid()))
print(finetuned.predict(['hello', 'hi'], activation_fct=nn.Identity()))

In [ ]:
# 미세 조정된 크로스 인코더 사용해보기
logger.setLevel(logging.CRITICAL)  # just to suppress some logs
from tqdm import tqdm

i = 0
print_every = 50
predictions = []
for question in tqdm(val_sample['question']):
    retrieved_hash, reranked_hash = eval_ranking(question, finetuned, top_k=3)
    correct_hash = q_to_hash[question]
    predictions.append((retrieved_hash == correct_hash, reranked_hash == correct_hash))
    i += 1
    if i % print_every == 0:
        print(f'Step {i}')
        raw_accuracy = sum([p[0] for p in predictions])/len(predictions)
        reranked_accuracy = sum([p[1] for p in predictions])/len(predictions)

        print(f'Accuracy without re-ranking: {raw_accuracy}')
        print(f'Accuracy with re-ranking: {reranked_accuracy}')


In [ ]:
# 재랭킹은 2번의 에포크 이후 약간 개선되었습니다.

In [ ]:
raw_accuracy = sum([p[0] for p in predictions])/len(predictions)
reranked_accuracy = sum([p[1] for p in predictions])/len(predictions)

print(f'Using cross-encoder: {finetuned.config._name_or_path}')
print(f'Accuracy without re-ranking: {raw_accuracy}')
print(f'Accuracy with re-ranking: {reranked_accuracy}')


In [ ]:
# pinecone.delete_index(INDEX_NAME)  # delete the index

# OPEN SOURCE ALTERNATIVE TO EMBEDDING

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-cos-v1')

docs = ["Around 9 Million people live in London", "London is known for its financial district"]

doc_emb = model.encode(docs, batch_size=32, show_progress_bar=True)

doc_emb.shape#  == ('2, 768')


In [ ]:
#쿼리 및 문서 인코딩
docs = dataset['validation']['passage']
doc_emb = model.encode(docs, batch_size=32, show_progress_bar=True)

In [ ]:
from random import sample

query = sample(dataset['validation']['question'], 1)[0]
print(query)
final_results = get_results_from_pinecone(query, top_k=3, re_rank=True)


In [ ]:
from sentence_transformers import util
query_emb = model.encode(query)

#쿼리와 모든 문서 임베딩 사이의 도트 점수를 계산합니다.
scores = util.dot_score(query_emb, doc_emb)[0].cpu().tolist()

#문서와 점수 결합
doc_score_pairs = list(zip(docs, scores))

#점수에 따른 내림차순 정렬
doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)

#구절 및 점수 출력
for doc, score in doc_score_pairs[:3]:
    print(score, doc)


In [ ]:
logger.setLevel(logging.CRITICAL)  # 일부 로그만 출력

def eval_ranking_open_source(query, cross_encoder, top_k=3):
    # query_emb = np.array(get_embedding(query))
    query_emb = model.encode(query)

    #쿼리와 모든 문서 임베딩 사이의 도트 점수를 계산합니다.
    scores = util.dot_score(query_emb, doc_emb)[0].cpu().tolist()

    #문서와 점수 결합
    doc_score_pairs = list(zip(docs, scores))

    #점수에 따른 내림차순 정렬
    doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)[:top_k]

    retrieved_hash = my_hash(doc_score_pairs[0][0])
    if cross_encoder:
        sentence_combinations = [[query, doc_score_pair[0]] for doc_score_pair in doc_score_pairs]
        similarity_scores = cross_encoder.predict(sentence_combinations)
        sim_scores_argsort = list(reversed(np.argsort(similarity_scores)))
        reranked_hash = my_hash(doc_score_pairs[sim_scores_argsort[0]][0])
    else:
        reranked_hash = None
    return retrieved_hash, reranked_hash


In [ ]:
eval_ranking_open_source(query, finetuned)

In [ ]:
logger.setLevel(logging.CRITICAL)

i = 0
print_every = 50
predictions = []
for question in tqdm(val_sample['question']):
    retrieved_hash, reranked_hash = eval_ranking_open_source(question, finetuned, top_k=3)
    correct_hash = q_to_hash[question]
    predictions.append((retrieved_hash == correct_hash, reranked_hash == correct_hash))
    i += 1
    if i % print_every == 0:
        print(f'Step {i}')
        raw_accuracy = sum([p[0] for p in predictions])/len(predictions)
        reranked_accuracy = sum([p[1] for p in predictions])/len(predictions)

        print(f'Accuracy without re-ranking: {raw_accuracy}')
        print(f'Accuracy with re-ranking: {reranked_accuracy}')


In [ ]:
raw_accuracy = sum([p[0] for p in predictions])/len(predictions)
reranked_accuracy = sum([p[1] for p in predictions])/len(predictions)

print(f'Using cross-encoder: {finetuned.config._name_or_path}')
print(f'Accuracy without re-ranking: {raw_accuracy}')
print(f'Accuracy with re-ranking: {reranked_accuracy}')
